<a href="https://colab.research.google.com/github/vsanupindi/climate-score/blob/master/lda_emi_preliminaryCC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
pip install pyLDAvis


In [0]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [0]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [15]:
# Import dataset
df = pd.read_csv('/content/gdrive/My Drive/ClimateTimes/for_emi.csv')
df.head()

,Unnamed: 0,id,title,author,date,content,year,month,publication,category,digital,section,url
0,134,151,Your meat addiction is destroying the planet,\nLaura June\n,2013-08-13,"&ampldquoJesus,&amprdquo Molly said, her...",2013.0,8.0,Verge,Longform,1.0,NaN,NaN
1,178,6467,Who should go to Mars?,Angela Chen,2016-09-30,Elon Musk laid out his plan to colonize ...,2016.0,9.0,Verge,Reports,1.0,NaN,NaN
2,180,6469,The Paris Motor Show opens amid a war on cars,Amar Toor,2016-09-29,"For a few fleeting hours last weekend, c...",2016.0,9.0,Verge,Reports,1.0,NaN,NaN
3,194,6483,Typhoons striking China and Southeast Asia hav...,Alessandra Potenza,2016-09-05,"Over the past 40 years, typhoons that st...",2016.0,9.0,Verge,Reports,1.0,NaN,NaN
4,195,6484,Even the basics of climate change are still be...,Lindsey J. Smith,2016-09-02,Between one Trump tweet after another an...,2016.0,9.0,Verge,Reports,1.0,NaN,NaN


In [0]:
# # Import Dataset
# df = pd.read_json('https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json')
# print(df.target_names.unique())
# df.head()

['rec.autos' 'comp.sys.mac.hardware' 'rec.motorcycles' 'misc.forsale'
 'comp.os.ms-windows.misc' 'alt.atheism' 'comp.graphics'
 'rec.sport.baseball' 'rec.sport.hockey' 'sci.electronics' 'sci.space'
 'talk.politics.misc' 'sci.med' 'talk.politics.mideast'
 'soc.religion.christian' 'comp.windows.x' 'comp.sys.ibm.pc.hardware'
 'talk.politics.guns' 'talk.religion.misc' 'sci.crypt']


,content,target,target_names
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7,rec.autos
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4,comp.sys.mac.hardware
10,From: irwin@cmptrc.lonestar.org (Irwin Arnstei...,8,rec.motorcycles
100,From: tchen@magnus.acs.ohio-state.edu (Tsung-K...,6,misc.forsale
1000,From: dabl2@nlm.nih.gov (Don A.B. Lindbergh)\n...,2,comp.os.ms-windows.misc


In [16]:
# Convert to list
data = df.content.values.tolist()

# Remove Emails
# data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', " ", sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

[' &ampldquoJesus,&amprdquo Molly said, her own plate empty, &ampldquogimme '
 'that. You know what this costs?&amprdquo She took his plate. &ampldquoThey '
 'gotta raise a whole animal for years and then they kill it. This '
 'isn&amprsquot vat stuff.&amprdquo She forked a mouthful up and '
 'chewed.&amprdquo &ampndash William Gibson, Neuromancer (1984)On Monday, '
 'August 5th, 2013, at a television studio in London in front of around 100 '
 'people, Dr. Mark Post of Maastricht University in the Netherlands unveiled '
 'the culmination of five years of research: a lab-grown &ampldquotest '
 'tube&amprdquo beef burger, cooked in a pan and served to two members of the '
 'public. Though a handful of tiny pieces of such meat had previously been '
 'displayed, the burger in that pan was the first fully cooked specimen tasted '
 'and admired by everyday citizens. &ampldquoA few cells that we take from a '
 'cow,&amprdquo Post says, can be turned into &ampldquo10 tons of '
 'meat.&amprdquo

In [17]:
pprint(data[0])

(' &ampldquoJesus,&amprdquo Molly said, her own plate empty, &ampldquogimme '
 'that. You know what this costs?&amprdquo She took his plate. &ampldquoThey '
 'gotta raise a whole animal for years and then they kill it. This '
 'isn&amprsquot vat stuff.&amprdquo She forked a mouthful up and '
 'chewed.&amprdquo &ampndash William Gibson, Neuromancer (1984)On Monday, '
 'August 5th, 2013, at a television studio in London in front of around 100 '
 'people, Dr. Mark Post of Maastricht University in the Netherlands unveiled '
 'the culmination of five years of research: a lab-grown &ampldquotest '
 'tube&amprdquo beef burger, cooked in a pan and served to two members of the '
 'public. Though a handful of tiny pieces of such meat had previously been '
 'displayed, the burger in that pan was the first fully cooked specimen tasted '
 'and admired by everyday citizens. &ampldquoA few cells that we take from a '
 'cow,&amprdquo Post says, can be turned into &ampldquo10 tons of '
 'meat.&amprdquo

In [18]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['ampldquojesus', 'amprdquo', 'molly', 'said', 'her', 'own', 'plate', 'empty', 'ampldquogimme', 'that', 'you', 'know', 'what', 'this', 'costs', 'amprdquo', 'she', 'took', 'his', 'plate', 'ampldquothey', 'gotta', 'raise', 'whole', 'animal', 'for', 'years', 'and', 'then', 'they', 'kill', 'it', 'this', 'isn', 'amprsquot', 'vat', 'stuff', 'amprdquo', 'she', 'forked', 'mouthful', 'up', 'and', 'chewed', 'amprdquo', 'ampndash', 'william', 'gibson', 'neuromancer', 'on', 'monday', 'august', 'th', 'at', 'television', 'studio', 'in', 'london', 'in', 'front', 'of', 'around', 'people', 'dr', 'mark', 'post', 'of', 'maastricht', 'university', 'in', 'the', 'netherlands', 'unveiled', 'the', 'culmination', 'of', 'five', 'years', 'of', 'research', 'lab', 'grown', 'ampldquotest', 'tube', 'amprdquo', 'beef', 'burger', 'cooked', 'in', 'pan', 'and', 'served', 'to', 'two', 'members', 'of', 'the', 'public', 'though', 'handful', 'of', 'tiny', 'pieces', 'of', 'such', 'meat', 'had', 'previously', 'been', 'displa

In [20]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['ampldquojesus', 'amprdquo', 'molly', 'said', 'her', 'own', 'plate', 'empty', 'ampldquogimme', 'that', 'you', 'know', 'what', 'this', 'costs', 'amprdquo', 'she', 'took', 'his', 'plate', 'ampldquothey', 'gotta', 'raise', 'whole', 'animal', 'for', 'years', 'and', 'then', 'they', 'kill', 'it', 'this', 'isn', 'amprsquot', 'vat', 'stuff', 'amprdquo', 'she', 'forked', 'mouthful', 'up', 'and', 'chewed', 'amprdquo', 'ampndash', 'william', 'gibson', 'neuromancer', 'on', 'monday', 'august', 'th', 'at', 'television', 'studio', 'in', 'london', 'in', 'front', 'of', 'around', 'people', 'dr', 'mark', 'post', 'of', 'maastricht', 'university', 'in', 'the', 'netherlands', 'unveiled', 'the', 'culmination', 'of', 'five', 'years', 'of', 'research', 'lab', 'grown', 'ampldquotest', 'tube', 'amprdquo', 'beef', 'burger', 'cooked', 'in', 'pan', 'and', 'served', 'to', 'two', 'members', 'of', 'the', 'public', 'though', 'handful', 'of', 'tiny', 'pieces', 'of', 'such', 'meat', 'had', 'previously', 'been', 'display

In [0]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [23]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

pprint(data_lemmatized[1])

['elon_musk',
 'lay',
 'plan',
 'colonize',
 'mar',
 'conference',
 'tuesday',
 'ampampa',
 'session',
 'woman',
 'ask',
 'key',
 'question',
 'choose',
 'embark',
 'risky',
 'trip',
 'colonize',
 'harsh',
 'planet',
 'normal',
 'people',
 'want',
 'travel',
 'mar',
 'need',
 'specific',
 'requirement',
 'ask',
 'normal',
 'people',
 'go',
 'spacex',
 'ceo',
 'answer',
 'line',
 'question',
 'try',
 'make',
 'anyone',
 'go',
 'say',
 'maybe',
 'day',
 'train',
 'however',
 'trip',
 'dangerous',
 'risk',
 'fatality',
 'really',
 'high',
 'way',
 'around',
 'later',
 'say',
 'would',
 'basically',
 'prepare',
 'die',
 'okay',
 'candidate',
 'go',
 'choose',
 'candidate',
 'mission',
 'mar',
 'may',
 'require',
 'bit',
 'send',
 'anyone',
 'space',
 'sound',
 'recipe',
 'disaster',
 'astronaut',
 'go',
 'low',
 'earth',
 'orbit',
 'pass',
 'strict',
 'vetting',
 'process',
 'live',
 'space',
 'confine',
 'environment',
 'high',
 'stress',
 'situation',
 'take',
 'physical',
 'mental',
 't

In [24]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 2), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 2), (11, 2), (12, 2), (13, 1), (14, 4), (15, 1), (16, 1), (17, 2), (18, 1), (19, 1), (20, 3), (21, 1), (22, 1), (23, 1), (24, 1), (25, 2), (26, 2), (27, 1), (28, 1), (29, 2), (30, 2), (31, 15), (32, 5), (33, 1), (34, 3), (35, 1), (36, 3), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 35), (48, 1), (49, 1), (50, 1), (51, 4), (52, 9), (53, 54), (54, 3), (55, 15), (56, 28), (57, 1), (58, 3), (59, 1), (60, 1), (61, 2), (62, 49), (63, 1), (64, 1), (65, 1), (66, 1), (67, 6), (68, 1), (69, 1), (70, 1), (71, 5), (72, 1), (73, 2), (74, 2), (75, 2), (76, 1), (77, 1), (78, 1), (79, 2), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 1), (86, 1), (87, 1), (88, 1), (89, 3), (90, 2), (91, 1), (92, 5), (93, 2), (94, 1), (95, 1), (96, 1), (97, 1), (98, 1), (99, 1), (100, 2), (101, 1), (102, 1), (103, 1), (104, 2), (105, 1), (106, 10), (107, 2), (108, 1), (109, 1), (

In [25]:
id2word[0]


'ability'

In [26]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('ability', 1),
  ('able', 1),
  ('absurdity', 1),
  ('abused', 1),
  ('accord', 2),
  ('achievable', 1),
  ('act', 1),
  ('activist', 1),
  ('activity', 1),
  ('actual', 1),
  ('actually', 2),
  ('add', 2),
  ('addition', 2),
  ('additive', 1),
  ('address', 4),
  ('admire', 1),
  ('adult', 1),
  ('advance', 2),
  ('advent', 1),
  ('adviser', 1),
  ('affect', 3),
  ('afford', 1),
  ('africa', 1),
  ('ago', 1),
  ('agothese', 1),
  ('agree', 2),
  ('agriculture', 2),
  ('airplane', 1),
  ('aisle', 1),
  ('allow', 2),
  ('already', 2),
  ('also', 15),
  ('alternative', 5),
  ('american', 1),
  ('amount', 3),
  ('amp', 1),
  ('amphellip', 3),
  ('ampldquo', 1),
  ('ampldquoa', 1),
  ('ampldquoalive', 1),
  ('ampldquogimme', 1),
  ('ampldquojesus', 1),
  ('ampldquoperfect', 1),
  ('ampldquoquite', 1),
  ('ampldquosome', 1),
  ('ampldquotaster', 1),
  ('ampldquotest', 1),
  ('ampmdash', 35),
  ('ampmdashwhich', 1),
  ('ampnbsp', 1),
  ('ampnbspput', 1),
  ('ampndash', 4),
  ('amprdquo', 

In [0]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [28]:
# Print the Keyword in the 10 topics
print(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0, '0.013*"say" + 0.012*"report" + 0.010*"also" + 0.010*"use" + 0.009*"find" + 0.008*"accord" + 0.008*"may" + 0.008*"study" + 0.007*"center" + 0.007*"environment"'), (1, '0.014*"dylan" + 0.008*"woman" + 0.007*"refugee" + 0.007*"attack" + 0.007*"russia" + 0.007*"syria" + 0.007*"terrorist" + 0.006*"security" + 0.006*"russian" + 0.006*"war"'), (2, '0.021*"pope" + 0.013*"fire" + 0.010*"there" + 0.009*"francis" + 0.009*"national_park" + 0.008*"minute" + 0.008*"gore" + 0.008*"jupiter" + 0.007*"tail" + 0.007*"home"'), (3, '0.036*"trump" + 0.023*"say" + 0.016*"president" + 0.013*"clinton" + 0.010*"obama" + 0.008*"campaign" + 0.008*"sander" + 0.007*"donald" + 0.007*"state" + 0.006*"american"'), (4, '0.022*"company" + 0.019*"product" + 0.017*"tax" + 0.015*"recall" + 0.014*"cookie_dough" + 0.010*"cost" + 0.010*"market" + 0.009*"may" + 0.009*"listeria" + 0.009*"bolund"'), (5, '0.039*"state" + 0.026*"california" + 0.023*"federal" + 0.016*"official" + 0.015*"law" + 0.014*"program" + 0.012*"project

In [29]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -8.385986435097388

Coherence Score:  0.43741793349353103


In [30]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/usr/local/lib/python3.6/dist-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
18    -0.286363  0.020599       1        1  20.506544
6     -0.245446 -0.060283       2        1  19.394737
3     -0.253333 -0.169156       3        1  18.601107
0     -0.150835  0.086579       4        1   6.073272
7     -0.106123  0.146870       5        1   4.331482
13    -0.138664 -0.026172       6        1   3.925361
1     -0.000416 -0.121288       7        1   3.859619
5     -0.053101 -0.008634       8        1   3.765331
17    -0.052977  0.224452       9        1   3.166635
11    -0.013744 -0.103390      10        1   2.398152
4      0.018286  0.128601      11        1   2.181042
12     0.137782  0.016427      12        1   1.950699
15     0.077574 -0.192332      13        1   1.768996
16     0.051562  0.143853      14        1   1.762352
14     0.147236  0.081410      15        1   1.649688
8      0.126752 -0.067761      16        1   1.164443
10     0.155418 -0.050699      17        1   1.042510
2      0.183577 -0.020472      18        1   1.010224
19     0.191972 -0.020404      19        1   0.967555
9      0.210843 -0.008199      20        1   0.480247, topic_info=      Category         Freq  ...  loglift  logprob
1864   Default  3880.000000  ...  30.0000  30.0000
1847   Default  2200.000000  ...  29.0000  29.0000
196    Default  2076.000000  ...  28.0000  28.0000
1813   Default  2156.000000  ...  27.0000  27.0000
879    Default  7853.000000  ...  26.0000  26.0000
1830   Default   796.000000  ...  25.0000  25.0000
195    Default  1359.000000  ...  24.0000  24.0000
3531   Default   713.000000  ...  23.0000  23.0000
1613   Default   915.000000  ...  22.0000  22.0000
1072   Default   702.000000  ...  21.0000  21.0000
1590   Default   424.000000  ...  20.0000  20.0000
1102   Default  2261.000000  ...  19.0000  19.0000
1717   Default  1309.000000  ...  18.0000  18.0000
701    Default  1479.000000  ...  17.0000  17.0000
3503   Default   488.000000  ...  16.0000  16.0000
739    Default  2273.000000  ...  15.0000  15.0000
2101   Default   429.000000  ...  14.0000  14.0000
1279   Default  1052.000000  ...  13.0000  13.0000
209    Default   729.000000  ...  12.0000  12.0000
1760   Default   642.000000  ...  11.0000  11.0000
2048   Default   601.000000  ...  10.0000  10.0000
1097   Default  1467.000000  ...   9.0000   9.0000
192    Default   538.000000  ...   8.0000   8.0000
1835   Default   578.000000  ...   7.0000   7.0000
1701   Default   866.000000  ...   6.0000   6.0000
244    Default  1506.000000  ...   5.0000   5.0000
4865   Default   349.000000  ...   4.0000   4.0000
335    Default   847.000000  ...   3.0000   3.0000
350    Default   467.000000  ...   2.0000   2.0000
3955   Default   771.000000  ...   1.0000   1.0000
...        ...          ...  ...      ...      ...
27522  Topic20    13.828282  ...   5.2678  -5.2335
4054   Topic20    12.733712  ...   5.2620  -5.3160
24064  Topic20    12.517355  ...   5.2607  -5.3331
3907   Topic20    12.238332  ...   5.2590  -5.3556
16331  Topic20    11.974540  ...   5.2573  -5.3774
18192  Topic20    11.974069  ...   5.2573  -5.3775
15228  Topic20    11.146617  ...   5.2515  -5.4491
9705   Topic20    10.785625  ...   5.2487  -5.4820
11389  Topic20    10.629575  ...   5.2475  -5.4966
3918   Topic20     9.691490  ...   5.2391  -5.5890
10075  Topic20     9.052312  ...   5.2324  -5.6572
4769   Topic20     8.657570  ...   5.2278  -5.7018
14718  Topic20     7.997561  ...   5.2192  -5.7811
8254   Topic20     7.979676  ...   5.2189  -5.7833
12975  Topic20     7.892682  ...   5.2177  -5.7943
24161  Topic20     7.594327  ...   5.2132  -5.8328
27519  Topic20     6.932308  ...   5.2020  -5.9240
12626  Topic20     6.864916  ...   5.2008  -5.9338
5471   Topic20     6.631028  ...   5.1963  -5.9684
3908   Topic20    21.845842  ...   5.1498  -4.7762
2127   Topic20    27.718981  ...   5.1098  -4.5381
7215   Topic20    55.052486  ...   5.0446  -3.8519
1590   Topic20 

In [0]:
# Download File: http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
mallet_path = '/content/gdrive/My Drive/ClimateTimes/mallet-2.0.8/bin/mallet' # update this path 
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


CalledProcessError: ignored

In [0]:
# Show Topics
print(ldamallet.show_topics(formatted=False))

# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

In [0]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [0]:
# Can take a long time to run.
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_lemmatized, start=2, limit=40, step=6)

In [0]:
# Show graph
limit=40; start=2; step=6;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

In [0]:
# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

In [0]:
Num Topics = 2  has Coherence Value of 0.4451
Num Topics = 8  has Coherence Value of 0.5943
Num Topics = 14  has Coherence Value of 0.6208
Num Topics = 20  has Coherence Value of 0.6438
Num Topics = 26  has Coherence Value of 0.643
Num Topics = 32  has Coherence Value of 0.6478
Num Topics = 38  has Coherence Value of 0.6525

In [0]:
# Select the model and print the topics
optimal_model = model_list[3]
model_topics = optimal_model.show_topics(formatted=False)
pprint(optimal_model.print_topics(num_words=10))

In [0]:
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


In [0]:
df_topic_sents_keywords = format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=data)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(10)

In [0]:
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]

# Show
sent_topics_sorteddf_mallet.head()

In [0]:
# Number of Documents for Each Topic
topic_counts = df_topic_sents_keywords['Dominant_Topic'].value_counts()

# Percentage of Documents for Each Topic
topic_contribution = round(topic_counts/topic_counts.sum(), 4)

# Topic Number and Keywords
topic_num_keywords = df_topic_sents_keywords[['Dominant_Topic', 'Topic_Keywords']]

# Concatenate Column wise
df_dominant_topics = pd.concat([topic_num_keywords, topic_counts, topic_contribution], axis=1)

# Change Column names
df_dominant_topics.columns = ['Dominant_Topic', 'Topic_Keywords', 'Num_Documents', 'Perc_Documents']

# Show
df_dominant_topics